In [31]:
import pandas as pd
import numpy as np
import torch
import optuna
import shap

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
from pytorch_tabnet.tab_model import TabNetClassifier


import matplotlib.pyplot as plt
%config InlineBackend.figure_format='svg'
plt.rcParams.update({
    'text.usetex':False,
    'font.family':'monospace'
})  
import seaborn as sns

Simple Bootstrapping method to get an confidence interval on the AUROC score.

In [32]:
def bootstrap_auc_ci(y_true, y_scores, n_bootstraps=2000, ci=0.95):
    rng = np.random.default_rng(42)
    aucs = []

    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    for _ in range(n_bootstraps):
        idx = rng.integers(0, len(y_true), len(y_true))
        if len(np.unique(y_true[idx])) < 2:
            continue
        aucs.append(roc_auc_score(y_true[idx], y_scores[idx]))

    lower = np.percentile(aucs, (1 - ci) / 2 * 100)
    upper = np.percentile(aucs, (1 + ci) / 2 * 100)
    return np.mean(aucs), lower, upper

## Loading the dataset, pre-processing, and analysing the data

In [33]:
cohort_data = pd.read_csv('../cohort_data_new.csv')
cohort_data

,icustay_id,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,...,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,gender,icu_los_hours,target
0,200003,13.375000,3.583195,9.0,21.0,25.250000,3.105295,18.0,28.0,7.771429,...,10.0,21.0,26.471429,13.176711,13.2,43.9,48,M,141,0
1,200007,15.500000,2.121320,14.0,17.0,23.000000,1.414214,22.0,24.0,8.900000,...,8.0,10.0,10.300000,1.272792,9.4,11.2,44,M,30,0
2,200009,9.500000,2.121320,8.0,11.0,23.333333,2.081666,21.0,25.0,8.000000,...,15.0,21.0,12.471429,1.471637,10.5,14.3,47,F,51,0
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.900000,NaN,4.9,4.9,33,F,10,0
4,200014,10.000000,1.732051,9.0,12.0,24.000000,1.000000,23.0,25.0,7.733333,...,21.0,24.0,13.233333,2.203028,10.7,14.7,85,M,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30484,299992,15.375000,2.856153,11.0,25.0,23.125000,2.609556,15.0,26.0,8.307143,...,8.0,23.0,14.134783,3.781727,8.1,22.1,41,M,499,0
30485,299993,9.400000,1.341641,8.0,11.0,29.600000,2.073644,26.0,31.0,8.000000,...,12.0,15.0,12.600000,0.605530,12.0,13.3,26,M,67,0
30486,299994,16.157895,2.477973,13.0,24.0,21.631579,3.451417,17.0,31.0,8.100000,...,28.0,63.0,10.076190,2.642329,5.3,14.5,74,F,152,1
30487,299998,11.500000,1.732051,10.0,14.0,23.500000,1.290994,22.0,25.0,8.800000,...,20.0,22.0,9.900000,1.210372,7.9,11.0,87,M,46,1


In [34]:
lab_cols = [
    'anion_gap_mean', 'anion_gap_min', 'anion_gap_max', 'anion_gap_sd',
    'bicarbonate_mean', 'bicarbonate_min', 'bicarbonate_max', 'bicarbonate_sd',
    'calcium_total_mean', 'calcium_total_min', 'calcium_total_max', 'calcium_total_sd',
    'chloride_mean', 'chloride_min', 'chloride_max', 'chloride_sd',
    'creatinine_mean', 'creatinine_min', 'creatinine_max', 'creatinine_sd',
    'glucose_mean', 'glucose_min', 'glucose_max', 'glucose_sd',
    'hematocrit_mean', 'hematocrit_min', 'hematocrit_max', 'hematocrit_sd',
    'hemoglobin_mean', 'hemoglobin_min', 'hemoglobin_max', 'hemoglobin_sd',
    'mchc_mean', 'mchc_min', 'mchc_max', 'mchc_sd',
    'mcv_mean', 'mcv_min', 'mcv_max', 'mcv_sd',
    'magnesium_mean', 'magnesium_min', 'magnesium_max', 'magnesium_sd',
    'pt_mean', 'pt_min', 'pt_max', 'pt_sd',
    'phosphate_mean', 'phosphate_min', 'phosphate_max', 'phosphate_sd',
    'platelet_count_mean', 'platelet_count_min', 'platelet_count_max', 'platelet_count_sd',
    'potassium_mean', 'potassium_min', 'potassium_max', 'potassium_sd',
    'rdw_mean', 'rdw_min', 'rdw_max', 'rdw_sd',
    'red_blood_cells_mean', 'red_blood_cells_min', 'red_blood_cells_max', 'red_blood_cells_sd',
    'sodium_mean', 'sodium_min', 'sodium_max', 'sodium_sd',
    'urea_nitrogen_mean', 'urea_nitrogen_min', 'urea_nitrogen_max', 'urea_nitrogen_sd',
    'white_blood_cells_mean', 'white_blood_cells_min', 'white_blood_cells_max', 'white_blood_cells_sd',
    'age', 'icu_los_hours'
]

In [35]:
# REmove the ICUstay_id and the gender
drop_cols = [c for c in cohort_data.columns if 'icustay_id' in c.lower() or 'gender' in c.lower()]
df = cohort_data.drop(columns=['icustay_id', 'gender'], errors='ignore')

X = df.drop(columns=['target'])
y = df['target']
imputer = SimpleImputer(strategy="median")
df[lab_cols] = imputer.fit_transform(df[lab_cols])

# Keep only numeric
X = X.select_dtypes(include=['number']).replace([np.inf, -np.inf], np.nan)

print(f"Final feature matrix shape: {X.shape}")

Final feature matrix shape: (30489, 94)


Simple Imputation to handle the missing/ NaN values

In [36]:
np.array(X)

array([[ 13.375     ,   3.5831949 ,   9.        , ...,  43.9       ,
         48.        , 141.        ],
       [ 15.5       ,   2.12132034,  14.        , ...,  11.2       ,
         44.        ,  30.        ],
       [  9.5       ,   2.12132034,   8.        , ...,  14.3       ,
         47.        ,  51.        ],
       ...,
       [ 16.15789474,   2.47797314,  13.        , ...,  14.5       ,
         74.        , 152.        ],
       [ 11.5       ,   1.73205081,  10.        , ...,  11.        ,
         87.        ,  46.        ],
       [         nan,          nan,          nan, ...,  20.7       ,
         49.        ,  31.        ]], shape=(30489, 94))

In [37]:
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['inr_pt__mean' 'inr_pt__sd' 'inr_pt__min' 'inr_pt__max']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


#### Splitting and creating the final datasets

In [38]:
# train-test
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.30, random_state=7)

# train-dev
X_val_train, X_val_test, y_val_train, y_val_test = train_test_split(X_test, y_test, test_size=0.30, random_state=42, stratify=y_test)

# Normalizing data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val_train = scaler.transform(X_val_train)
X_val_test = scaler.transform(X_val_test)

print(f'% readmission in train: {np.mean(y_val_train)*100}')
print(f'% readmission in train: {np.mean(y_val_test)*100}')
print(f'% readmission in train: {np.mean(y_test)*100}')

% readmission in train: 10.731021555763824
% readmission in train: 10.746812386156648
% readmission in train: 10.735760358587514


Hyperparameter Tuning

In [39]:
def objective(trial):
    params = {
        "n_d": trial.suggest_int("n_d", 16, 64),
        "n_a": trial.suggest_int("n_a", 16, 64),
        "n_steps": trial.suggest_int("n_steps", 3, 7),
        "gamma": trial.suggest_float("gamma", 1.0, 2.5),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True),
        "optimizer_fn": torch.optim.Adam,
        "optimizer_params": dict(lr=trial.suggest_float("lr", 1e-4, 3e-3, log=True)),
        "mask_type": "sparsemax",
    }

    model = TabNetClassifier(**params)

    model.fit(X_train, y_train, eval_set=[(X_val_train, y_val_train), (X_val_test, y_val_test)],
        eval_metric=["auc"],
        max_epochs=150,
        patience=15,
        batch_size=2048,
        virtual_batch_size=256,
        num_workers=0,
        drop_last=False,
    )

    # Predict on validation test split
    val_pred_proba = model.predict_proba(X_val_test)[:, 1]
    auc = roc_auc_score(y_val_test, val_pred_proba)

    return auc

In [40]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Best AUC:", study.best_value)
print("Best parameters:", study.best_params)


[I 2025-11-28 12:07:35,788] A new study created in memory with name: no-name-9d351a4a-700c-4e4f-b093-a080cc3fd87f


  0%|          | 0/40 [00:00<?, ?it/s]

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.31105 | val_0_auc: 0.47387 | val_1_auc: 0.49133 |  0:00:06s
epoch 1  | loss: 1.03798 | val_0_auc: 0.47567 | val_1_auc: 0.48662 |  0:00:14s
epoch 2  | loss: 0.84756 | val_0_auc: 0.46776 | val_1_auc: 0.4838  |  0:00:23s
epoch 3  | loss: 0.7183  | val_0_auc: 0.45989 | val_1_auc: 0.47143 |  0:00:32s
epoch 4  | loss: 0.65099 | val_0_auc: 0.45906 | val_1_auc: 0.47001 |  0:00:41s
epoch 5  | loss: 0.61761 | val_0_auc: 0.47248 | val_1_auc: 0.45602 |  0:00:49s
epoch 6  | loss: 0.58399 | val_0_auc: 0.46051 | val_1_auc: 0.49064 |  0:00:57s
epoch 7  | loss: 0.59339 | val_0_auc: 0.46907 | val_1_auc: 0.4677  |  0:01:05s
epoch 8  | loss: 0.57427 | val_0_auc: 0.46097 | val_1_auc: 0.48063 |  0:01:14s
epoch 9  | loss: 0.56195 | val_0_auc: 0.47132 | val_1_auc: 0.48681 |  0:01:24s
epoch 10 | loss: 0.5536  | val_0_auc: 0.47436 | val_1_auc: 0.49062 |  0:01:32s
epoch 11 | loss: 0.54995 | val_0_auc: 0.49165 | val_1_auc: 0.49974 |  0:01:42s
epoch 12 | loss: 0.54118 | val_0_auc: 0.49769 | val_

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

[I 2025-11-28 15:16:55,004] Trial 38 finished with value: 0.6428073331027326 and parameters: {'n_d': 40, 'n_a': 42, 'n_steps': 7, 'gamma': 1.4761637107025187, 'lambda_sparse': 9.462659789248206e-05, 'lr': 0.0014249095974565842}. Best is trial 17 with value: 0.6897391905914908.


c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.54143 | val_0_auc: 0.4836  | val_1_auc: 0.47725 |  0:00:02s
epoch 1  | loss: 0.49694 | val_0_auc: 0.48019 | val_1_auc: 0.48713 |  0:00:04s
epoch 2  | loss: 0.47681 | val_0_auc: 0.48516 | val_1_auc: 0.48924 |  0:00:07s
epoch 3  | loss: 0.45545 | val_0_auc: 0.4936  | val_1_auc: 0.49248 |  0:00:10s
epoch 4  | loss: 0.44313 | val_0_auc: 0.4934  | val_1_auc: 0.50318 |  0:00:12s
epoch 5  | loss: 0.42936 | val_0_auc: 0.49748 | val_1_auc: 0.501   |  0:00:15s
epoch 6  | loss: 0.41191 | val_0_auc: 0.50362 | val_1_auc: 0.51451 |  0:00:17s
epoch 7  | loss: 0.41188 | val_0_auc: 0.51492 | val_1_auc: 0.51886 |  0:00:20s
epoch 8  | loss: 0.40432 | val_0_auc: 0.52072 | val_1_auc: 0.51843 |  0:00:22s
epoch 9  | loss: 0.3929  | val_0_auc: 0.52452 | val_1_auc: 0.51495 |  0:00:25s
epoch 10 | loss: 0.38468 | val_0_auc: 0.5297  | val_1_auc: 0.52007 |  0:00:28s
epoch 11 | loss: 0.38017 | val_0_auc: 0.53582 | val_1_auc: 0.53679 |  0:00:30s
epoch 12 | loss: 0.37825 | val_0_auc: 0.54668 | val_

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[I 2025-11-28 15:21:18,850] Trial 39 finished with value: 0.6480837080594949 and parameters: {'n_d': 25, 'n_a': 24, 'n_steps': 3, 'gamma': 1.1064166425359885, 'lambda_sparse': 1.037615512861142e-06, 'lr': 0.0008666777106614362}. Best is trial 17 with value: 0.6897391905914908.
Best AUC: 0.6897391905914908
Best parameters: {'n_d': 37, 'n_a': 44, 'n_steps': 5, 'gamma': 1.020080536443378, 'lambda_sparse': 0.00017353660543982074, 'lr': 0.0028875987502131164}


Final retraining using the best hyperparameters

In [42]:
best_params = study.best_params

final_model = TabNetClassifier(
    n_d=best_params["n_d"],
    n_a=best_params["n_a"],
    n_steps=best_params["n_steps"],
    gamma=best_params["gamma"],
    lambda_sparse=best_params["lambda_sparse"],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params["lr"]),
    mask_type="sparsemax",
)

final_model.fit(
    X_train, y_train,
    eval_set=[(X_val_train, y_val_train), (X_val_test, y_val_test)],
    eval_metric=["auc"],
    max_epochs=200,
    patience=20,
    batch_size=2048,
    virtual_batch_size=256,
)

epoch 0  | loss: 0.57335 | val_0_auc: 0.46297 | val_1_auc: 0.47098 |  0:00:03s
epoch 1  | loss: 0.50533 | val_0_auc: 0.49664 | val_1_auc: 0.49988 |  0:00:06s
epoch 2  | loss: 0.46029 | val_0_auc: 0.51098 | val_1_auc: 0.4882  |  0:00:09s
epoch 3  | loss: 0.41973 | val_0_auc: 0.52857 | val_1_auc: 0.49668 |  0:00:12s
epoch 4  | loss: 0.40631 | val_0_auc: 0.55205 | val_1_auc: 0.52753 |  0:00:15s
epoch 5  | loss: 0.39626 | val_0_auc: 0.54954 | val_1_auc: 0.53482 |  0:00:18s
epoch 6  | loss: 0.38969 | val_0_auc: 0.56262 | val_1_auc: 0.55447 |  0:00:22s
epoch 7  | loss: 0.37203 | val_0_auc: 0.5545  | val_1_auc: 0.55976 |  0:00:26s
epoch 8  | loss: 0.36973 | val_0_auc: 0.56375 | val_1_auc: 0.56695 |  0:00:30s
epoch 9  | loss: 0.36433 | val_0_auc: 0.597   | val_1_auc: 0.58885 |  0:00:35s
epoch 10 | loss: 0.35547 | val_0_auc: 0.59289 | val_1_auc: 0.57147 |  0:00:40s
epoch 11 | loss: 0.35374 | val_0_auc: 0.58789 | val_1_auc: 0.60863 |  0:00:45s
epoch 12 | loss: 0.34972 | val_0_auc: 0.60563 | val_

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Evaluation

In [48]:
X_test_final = scaler.transform(X_test)

proba = final_model.predict_proba(X_test_final)[:, 1]
preds = final_model.predict(X_test_final)

print("Final Test AUC:", roc_auc_score(y_test, proba))
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

Final Test AUC: 0.6642736432764657
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      8165
           1       0.18      0.00      0.00       982

    accuracy                           0.89      9147
   macro avg       0.54      0.50      0.47      9147
weighted avg       0.82      0.89      0.84      9147

